In [160]:
import sys

import joblib

import click
import progressbar
import pyfaidx
import pandas as pd
from gtfparse import read_gtf

In [109]:
gtffile = "/Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.94.gtf.gz"
fastafile = "/Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa"
feature_type = "gene"
outfile = "/Users/milessmith/workspace/merrycrispr/merrycrispr/data/test.fasta"
gene_name = ["PML"]
# gene_name = None
boundary = 0

In [55]:
records = read_gtf(gtffile)
if gene_name:
    records = records[records['gene_name'].isin(gene_name)]
if feature_type:
    records = records[records['feature'] == feature_type]
    records = records[['seqname', 
                       'feature', 
                       'start', 
                       'end', 
                       'strand', 
                       'frame', 
                       'gene_name', 
                       f'{feature_type}_id']].drop_duplicates()
else:
    records = records[['seqname', 'feature', 'start', 'end', 'strand', 'frame', 'gene_name']].drop_duplicates()

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'protein_id', 'protein_version', 'ccds_id']


In [111]:
print(f"{len(records)} total records found.")

print(f"Loading the sequences in {fastafile}.  "
      f"Note: if this is the first time opening this file, "
      "it may take a few moments as an index is built.")
sequences = pyfaidx.Fasta(fastafile)
print(f"Finished loading {fastafile}")

1 total records found.
Loading the sequences in /Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa.  Note: if this is the first time opening this file, it may take a few moments as an index is built.
Finished loading /Users/milessmith/workspace/ensembl/Homo_sapiens.GRCh38.dna.primary_assembly.fa


In [113]:
records.iloc[0]

seqname                   15
feature                 gene
start               73994673
end                 74047812
strand                     +
frame                      0
gene_name                PML
gene_id      ENSG00000140464
Name: 2027769, dtype: object

In [112]:
sequences.get_seq("15", records.iloc[0].start, records.iloc[0].end)

>15:73994673-74047812
...

In [130]:
sequences.get_seq("15", 100000000, 100000100, rc=False)

>15:100000000-100000100
GCAGGGATGGGAGCCACAGTGACTCCCTTGGGGTGCAGTTGTGAGGACAAGTGGCACATTGTCCATAAACTGCTTACAACAGTGTCTGGCACACCTGAAGC

In [131]:
sequences.get_seq("15", 100000000, 100000100, rc=True)

>15:100000100-100000000 (complement)
GCTTCAGGTGTGCCAGACACTGTTGTAAGCAGTTTATGGACAATGTGCCACTTGTCCTCACAACTGCACCCCAAGGGAGTCACTGTGGCTCCCATCCCTGC

In [132]:
rc=True
sequences.get_seq("15", 100000000, 100000100, rc=rc)

>15:100000100-100000000 (complement)
GCTTCAGGTGTGCCAGACACTGTTGTAAGCAGTTTATGGACAATGTGCCACTTGTCCTCACAACTGCACCCCAAGGGAGTCACTGTGGCTCCCATCCCTGC

In [96]:
rec1_sequence = sequences.get_seq("15", records.iloc[0].start, records.iloc[0].end)
type(rec1_sequence)

pyfaidx.Sequence

In [105]:
sequences["15"][records.iloc[0].start:records.iloc[0].end]

>15:73994674-74047812
...

In [57]:
seq_count = 1
seq_widgets = ['Matching features to sequences: ', 
               progressbar.Counter(),
               f'/{records.index}',
               progressbar.Percentage(),
               ' ',
               progressbar.Bar(),
               progressbar.Timer(),
               ' ',
               progressbar.ETA()]
seq_progress = progressbar.ProgressBar(widgets=seq_widgets,
                                       maxval=len(records.index)).start()

In [58]:
records.gene_name

2027769    PML
Name: gene_name, dtype: object

In [59]:
records.start

2027769    73994673
Name: start, dtype: int64

In [60]:
type(boundary)

int

In [16]:
seq.name

'PML_ENSG00000140464_+_73994673_74047812'

In [17]:
from copy import deepcopy

In [18]:
seq2 = deepcopy(seq)

In [19]:
seq2.name

'PML_ENSG00000140464_+_73994673_74047812'

In [20]:
seq2.name = seq2.name + "_upstream"

In [114]:
help(sequences.get_seq)

Help on method get_seq in module pyfaidx:

get_seq(name, start, end, rc=False) method of pyfaidx.Fasta instance
    Return a sequence by record name and interval [start, end).
    
    Coordinates are 1-based, end-exclusive.
    If rc is set, reverse complement will be returned.



In [21]:
seq2.name

'PML_ENSG00000140464_+_73994673_74047812_upstream'

In [170]:
rec = records.iloc[0]

In [171]:
rec

seqname                   15
feature                 gene
start               73994673
end                 74047812
strand                     +
frame                      0
gene_name                PML
gene_id      ENSG00000140464
Name: 2027769, dtype: object

In [158]:
from typing import Tuple

In [174]:
from copy import deepcopy
rec1 = deepcopy(rec)
rec2 = deepcopy(rec)

In [173]:
def split_record(rec: pd.Series, 
                 padding: int,
                 rev: bool=False) -> Tuple[pd.Series, pd.Series]:
    rec1 = deepcopy(rec)
    rec2 = deepcopy(rec)
    if rev == False:
        rec1.end = rec1.start
        rec1.start -= padding
        rec1.gene_name += "_upstream"
        rec2.start = rec2.end
        rec2.end += padding
        rec2.gene_name += "_downstream"
    elif rev == True:
        rec1.end = rec1.start
        rec1.start -= padding
        rec1.gene_name += "_downstream"
        rec2.start = rec2.end
        rec2.end += padding
        rec2.gene_name += "_upstream"
    return rec1, rec2

In [175]:
split_record(rec, 100)

AttributeError: __enter__

In [177]:
type(sequences)

pyfaidx.Fasta

In [143]:
def match_seq(rec, sequences):
    try:
        if rec.strand == "-":
            rc = True
        else:
            rc = False
        seq = pyfaidx.Sequence(name=f"{rec.gene_name}_{rec.feature}_"
                                    f"{rec.strand}_{rec.start}_{rec.end}",
            a
                               seq=sequences.get_seq(name=rec.seqname, 
                                                     start=rec.start, 
                                                     end=rec.end, 
                                                     rc=rc).seq)
    except ValueError:
        print(f"problem with {rec.gene_name} {rec.start} "
              f"{rec.end} {rec.seqname} {rec.strand}")
    return seq

In [145]:
match_seq(rec=rec, sequences=sequences)

>PML_gene_+_73994973_74047812
...

In [154]:
alpha = [match_seq(_, sequences) for _ in records.itertuples()]

In [22]:
final_list = []
Parallel(n_jobs=2, prefer="threads")(
    for rec in records.itertuples():
        seq_progress.update(seq_count)
        seq_count += 1

        
)
    seq_progress.finish()

SyntaxError: invalid syntax (<ipython-input-22-7d0c99d8d29d>, line 3)

In [24]:
with open(outfile, 'w') as o_file:
    for entry in final_list:
        o_file.writelines(f"> {entry.fancy_name}\n{entry.seq}\n")